# 03: Projections

It's hard to visualize high-dimensional data. There's a limited number of channels that we can use at once to encode the dimensions. One approach is to project the high-dimensional data into fewer dimensions, often 2. There are many techniques for doing this, such as t-SNE, UMAP, PCA, and locally linear embeddings. Let's see how projections can be useful for exploring machine learning datasets and model predictions.

## Imports

In [ ]:
# If you're running this on colab, then you can uncomment the below command to
# install the pmlb library.
# !pip install pmlb

In [ ]:
import altair as alt
import numpy as np
import pandas as pd
import pmlb
from itertools import product

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits

from PIL import Image

In [ ]:
# If you're running this code locally, then you can uncomment this to automatically
# save the chart data in files, rather than including the data in the spec. 

!mkdir -p data
alt.data_transformers.enable('json', prefix='data/altair-data')

## Data Preparation and t-SNE

Load the dataset and take a random sample of it.

In [ ]:
mnist = pmlb.fetch_data('mnist')

In [ ]:
mnist_small = mnist.sample(n=5000)

Separate the feature values from the target labels. Split the dataset into train and test sets.

In [ ]:
X = mnist_small.drop(columns=['target'])
y = mnist_small['target'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

Create 2D projections of the dataset.

In [ ]:
tsne = TSNE(n_components=2, learning_rate='auto', init='random', perplexity=3)
X_train_embedded = tsne.fit_transform(X_train)
X_test_embedded = tsne.fit_transform(X_test)

Save the instances as image files.

In [ ]:
def save_images(matrix, prefix):
    paths = []
    
    for i, row in enumerate(matrix):
        path = f'digits/{prefix}-{i}.png'
        
        d = int(np.sqrt(len(row)))
        
        img = row.reshape((d, d)).astype(np.uint8)
        
        Image.fromarray(img).save(path)
        paths.append(path)
        
    return paths

In [ ]:
!mkdir -p digits

images_train = save_images(X_train.values, 'train')
images_test = save_images(X_test.values, 'test')

Prepare the data frames for visualizaiton.

In [ ]:
df_train = pd.DataFrame(X_train_embedded, columns=['x-tsne', 'y-tsne'])
df_train['target'] = y_train
df_train['image'] = images_train

df_train.head()

In [ ]:
df_test = pd.DataFrame(X_test_embedded, columns=['x-tsne', 'y-tsne'])
df_test['target'] = y_test
df_test['image'] = images_test

df_test.head()

## Visualizing Training Data

**Exercise 1:** Create a scatterplot of the projection in `df_train`. What would make this scatterplot more useful?

In [ ]:
alt.Chart(df_train).mark_circle().encode(
    x='x-tsne',
    y='y-tsne',
)

**Exercise 2:** Replace the circles with digits. Is this effective?

In [ ]:
alt.Chart(df_train).mark_text().encode(
    x='x-tsne',
    y='y-tsne',
    text='target'
)

**Exercise 3:** Color the circles by their target label. Make the scatterplot support panning and zooming. Do you notice anything interesting? What would make this visualization more useful?

In [ ]:
alt.Chart(df_train).mark_circle().encode(
    x='x-tsne',
    y='y-tsne',
    color='target:N'
).interactive()

**Exercise 4:** Add a tooltip to the visualization that shows the image and the target label. Does this lead to any interesting findings?

In [ ]:
alt.Chart(df_train).mark_circle().encode(
    x='x-tsne',
    y='y-tsne',
    color='target:N',
    tooltip=['image', 'target']
).interactive()

**Exercise 5:** Follow this [example](https://altair-viz.github.io/user_guide/marks.html#image-mark) of the `mark_image` to create a scatterplot that shows the images rather than circles. You'll likely only want to show a subset of the data. Do you find this visualization more or less useful than the previous one?

In [ ]:
alt.Chart(df_train.sample(500)).mark_image(width=14, height=14).encode(
    x='x-tsne',
    y='y-tsne',
    url='image'
).interactive()

Let's add another projection to compare to t-SNE.

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
X_train_pca = pca.fit_transform(X_train)

df_train['x-pca'] = X_train_pca[:,0]
df_train['y-pca'] = X_train_pca[:,1]

In [ ]:
alt.Chart(df_train).mark_circle().encode(
    x='x-pca',
    y='y-pca',
    color='target:N',
    tooltip=['image', 'target']
).interactive()

**Exercise 6:** Create side-by-side scatterplots for the t-SNE projection and the PCA projection.

In [ ]:
base = alt.Chart(df_train).mark_circle().encode(
    color='target:N',
).properties(
    width=350,
    height=350
)

base.encode(x='x-tsne', y='y-tsne') | base.encode(x='x-pca', y='y-pca')

**Exercise 7:** Add a brush selection to the scatterplots so that selecting points in one of the scatterplots highlights them in the other.

In [ ]:
brush = alt.selection_interval()

base = alt.Chart(df_train).mark_circle().encode(
    color=alt.condition(brush, 'target:N', alt.value('#dddddd')),
).add_selection(
    brush
).properties(
    width=350,
    height=350
)

base.encode(x='x-tsne', y='y-tsne') | base.encode(x='x-pca', y='y-pca')

## Modeling

**Exercise 8:** Train a model on this dataset. You can use [scikit-learn](https://scikit-learn.org/stable/) or any library of your choosing. Save the predictions on the test dataset in `df_test`.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
param_grid = {
    'n_neighbors': [2, 4, 8, 16, 32, 64, 128],
    'weights': ['uniform', 'distance'],
    'algorithm': ['ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2]
}

cv = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid, scoring='accuracy', n_jobs=-1)

cv.fit(X_train, y_train)

In [ ]:
cv.best_params_

In [ ]:
cv.best_score_

In [ ]:
model = cv.best_estimator_

In [ ]:
df_test['prediction'] = model.predict(X_test)

**Exercise 9:** Create a confusion matrix. What are the most common mistakes that the model makes?

In [ ]:
cm = df_test.groupby(['target', 'prediction'], as_index=False).size()
cm

In [ ]:
base = alt.Chart(cm).encode(
    x=alt.X('prediction:N', title='predicted class', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('target:N', title='actual class'),
).properties(
    width=300,
    height=300
)

rect = base.mark_rect().encode(
    color=alt.Color('prediction:N', legend=None),
    opacity=alt.Opacity('size', legend=None),
)

text = base.mark_text().encode(
    text='size',
)

rect + text

Confusion matrix with zeros \[[1](https://stackoverflow.com/a/42854842/5016634)\].

In [ ]:
cm_with_zeros = df_test.groupby(['target', 'prediction']).size().unstack(fill_value=0).stack().reset_index()
cm_with_zeros.rename(columns={0: 'size'}, inplace=True)
cm_with_zeros

In [ ]:
base = alt.Chart(cm_with_zeros).encode(
    x=alt.X('prediction:N', title='predicted class', axis=alt.Axis(labelAngle=0)),
    y=alt.Y('target:N', title='actual class'),
).properties(
    width=300,
    height=300
)

rect = base.mark_rect().encode(
    color=alt.Color('prediction:N', legend=None),
    opacity=alt.Opacity('size', scale=alt.Scale(range=[0, 1], type='symlog'), legend=None),
)

text = base.mark_text().encode(
    text='size',
)

rect + text

**Exercise 10:** Create a scatterplot that shows the t-SNE projection of the test data. How can the scatteplot be made useful for exploring incorrectly classified points? Can you show which points were correctly classified vs. incorrectly classified? Can you provide a way for the user to see both the target and predicted labels?

In [ ]:
df_test['correct'] = df_test['target'] == df_test['prediction']

In [ ]:
alt.Chart(df_test).mark_circle().encode(
    x='x-tsne',
    y='y-tsne',
    stroke='target:N',
    fill='prediction:N',
    opacity=alt.condition(alt.datum.correct, alt.value(0.15), alt.value(1)),
    size=alt.condition(alt.datum.correct, alt.value(25), alt.value(50)),
    order=alt.Order('correct', sort='descending'),
    tooltip=['image', 'target', 'prediction']
).interactive()